In [ ]:
import torch
import numpy as np
import os
import sys
sys.path.append('../')
from ood_gan import *
from models.dc_gan_model import *
from dataset import *
from config import *
from models.model import *
from eval import *
import argparse
import time
import yaml

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
G = PCGenerator().to(device)
path = "..\\checkpoint\\OOD-GAN\\3DPC\\Imbalanced\\50\\[3DPC]-[50]-[Imbalanced]-[0]_[3].pt"
ckpt = torch.load(path, map_location=device)
G.load_state_dict(torch.load(path, map_location=device)['G-state'])

cuda


<All keys matched successfully>

In [ ]:
z = torch.normal(0, 1, (1, 32)).to(device)
img = G(z).detach().squeeze().cpu().numpy()
print(img.shape)

(300, 3)


In [45]:
# visualize 3D point cloud
import open3d as o3d
import matplotlib.pyplot as plt
# Visualize the point cloud using Open3D
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(img)
# pcd.colors = o3d.utility.Vector3dVector(np.random.rand(img.shape[0], 3))  # Optional: Add random colors to the points

o3d.visualization.draw_geometries([pcd])


In [41]:
path = "../checkpoint/OOD-Sample/3DPC/OOD-Imbalanced-1000.pt"
data = torch.load(path)
data = data[0]
print(data.shape)

eg = data[0].unsqueeze(0).squeeze().cpu().numpy()
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(eg)
# pcd.colors = o3d.utility.Vector3dVector(np.random.rand(img.shape[0], 3))  # Optional: Add random colors to the points

o3d.visualization.draw_geometries([pcd])


torch.Size([1000, 1, 300, 3])
